In [1]:
import torch
import torch.nn as nn
from ignite.engine.engine import Engine,Events
from ignite.metrics import RunningAverage,Accuracy,Loss
from ignite.handlers import ModelCheckpoint,EarlyStopping
from ignite.contrib.handlers import ProgressBar

In [2]:
import sys
if '../' not in sys.path:
    sys.path.append('../')

import torch.nn.functional as F
import sys
import os
import numpy as np
from libnlp import preprocessing,env,custData
import pandas as pd
from torch.utils.data import DataLoader,Dataset
from ignite.utils import convert_tensor
from pytorch_pretrained_bert.modeling import BertModel

Using cache found in C:\Users\Jamiu Afolabi/.cache\torch\hub\huggingface_pytorch-transformers_master
Using cache found in C:\Users\Jamiu Afolabi/.cache\torch\hub\huggingface_pytorch-transformers_master


In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim import AdamW
from tqdm import tqdm
from argparse import ArgumentParser
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.engine.engine import Engine, State, Events
from ignite.handlers import EarlyStopping
from ignite.contrib.handlers import TensorboardLogger, ProgressBar
from ignite.utils import convert_tensor
from torch.optim.lr_scheduler import ExponentialLR
import warnings  
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

In [4]:
E=env.Env()

Using cache found in C:\Users\Jamiu Afolabi/.cache\torch\hub\huggingface_pytorch-transformers_master
Using cache found in C:\Users\Jamiu Afolabi/.cache\torch\hub\huggingface_pytorch-transformers_master


In [5]:
train_df=pd.read_csv(os.path.join(E.ppd_path,E.train_path),encoding='utf-8')
test_df=pd.read_csv(os.path.join(E.ppd_path,E.test_path),encoding='utf-8')
train_minority_df=pd.read_csv(os.path.join(E.ppd_path,'minority_train.csv'),encoding='utf-8')
train_minority_df_idx=train_minority_df.index
validation_minority_df=pd.read_csv(os.path.join(E.ppd_path,'minority_test.csv'),encoding='utf-8')
validation_minority_df_idx=train_minority_df.index
train_df=train_df[train_df.label!=0]
#train_data=pd.concat([train_df,train_minority_df])

In [6]:
train_minority_df_idx

RangeIndex(start=0, stop=8164, step=1)

# Split Train_df into two

In [7]:
train_one,train_two=train_test_split(train_df,test_size=0.5)

# Create Datasets and Using Train_one

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

def createDataLoaders(train,test,train_minority,validation_minority):
    train_s,validation_s=train_test_split(train,test_size=0.2,random_state=42)
    train_s=pd.concat([train_s,train_minority],axis=0)
    train_s=shuffle(train_s,random_state=42)
    validation_s=pd.concat([validation_s,validation_minority],axis=0)
    validation_s=shuffle(validation_s,random_state=42)
    train_ds=custData.ArabiziDataset(train_s,E.tokenizer,E.tokenizer_max_len)
    validation_ds=custData.ArabiziDataset(validation_s,E.tokenizer,E.tokenizer_max_len)
    test_ds=custData.ArabiziDataset(test_df,E.tokenizer,E.tokenizer_max_len,train=False)
    train_loader=DataLoader(train_ds,batch_size=32,collate_fn=custData.customPadding,shuffle=False)
    validation_loader=DataLoader(validation_ds,batch_size=32,collate_fn=custData.customPadding,shuffle=False)
    test_loader=DataLoader(test_ds,batch_size=256,collate_fn=custData.customPadding,shuffle=False)
    
    return train_loader,validation_loader,test_loader

# Using the whole train_df

In [9]:
train_loader,validation_loader,test_loader=createDataLoaders(train_two,test_df,train_minority_df,validation_minority_df)

# Create a Model

In [10]:
class BertModel(nn.Module):
    def __init__(self,n_outputs,bert_model=E.model,num_filters=200,kernel_size=[3,4,5],d_prob=0.2):
        super(BertModel,self).__init__()
        self.K=n_outputs
        self.bert_model=E.model
        self.bert_hidden_size=E.model.config.hidden_size
        self.conv=nn.ModuleList([nn.Conv1d(in_channels=self.bert_hidden_size,
                                           out_channels=num_filters,
                                           kernel_size=k,
                                           stride=1
                                          ) for k in kernel_size])
        self.dropout=nn.Dropout(d_prob)
        self.fc=nn.Linear(len(kernel_size)*num_filters,self.K)
        
        
    def forward(self,input_tokens,attention_mask):
        out,_=self.bert_model(input_ids=input_tokens,attention_mask=attention_mask,return_dict=False)
        out=out.permute(0,2,1)
        out=[F.relu(conv(out)) for conv in self.conv]
        out=[F.max_pool1d(c,c.size(-1)).squeeze(dim=-1) for c in out]
        out=torch.cat(out,dim=1)
        out=self.fc(self.dropout(out))
        return out
        

In [11]:
def children(m):
    return m if isinstance(m, (list, tuple)) else list(m.children())


def set_trainable_attr(m, b):
    m.trainable = b
    for p in m.parameters():
        p.requires_grad = b


def apply_leaf(m, f):
    c = children(m)
    if isinstance(m, nn.Module):
        f(m)
    if len(c) > 0:
        for l in c:
            apply_leaf(l, f)

            
def set_trainable(l, b):
    apply_leaf(l, lambda m: set_trainable_attr(m, b))

In [12]:
model=BertModel(3)
model.to(E.device)
set_trainable(model.bert_model,False)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters())

# Building helper to Run Model

In [13]:
def _batch(batch,train=True):
    if train:
        input_token,attention_mask,targets=batch
        return input_token.to(E.device),attention_mask.to(E.device),targets.to(E.device)
    else:
        input_token,attention_mask=batch
        return input_token.to(E.device),attention_mask.to(E.device)

def trainer_update(engine,batch):
    model.train()
    optimizer.zero_grad()
    input_token,attention_mask,y=_batch(batch)
    #Forward pass
    out=model(input_token,attention_mask)
    loss=criterion(out,y)
    #backward pass
    loss.backward()
    optimizer.step()
    #print('Done')
    return loss.item()

def eval_update(engine,batch):
    model.eval()
    with torch.no_grad():
        input_token,attention_mask,y=_batch(batch)
        out=model(input_token,attention_mask)
        y_pred=torch.round(out)
        return y_pred,y
    
# Creating the Engine
trainer=Engine(trainer_update)
train_evaluator=Engine(eval_update)
RunningAverage(output_transform=lambda x: x).attach(trainer,'loss')
Loss(criterion).attach(train_evaluator,'crossentropy')
Accuracy().attach(train_evaluator,'accuracy')

# Progress bar
pbar=ProgressBar(persist=True,bar_format="")
pbar.attach(trainer,['loss'])

# Tracking Validation Loss
def score_func(engine):
    val_loss=engine.state.metrics['crossentropy']
    return -val_loss

handler=EarlyStopping(patience=3,score_function=score_func,trainer=trainer)
train_evaluator.add_event_handler(Events.COMPLETED,handler)

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_res():
    train_evaluator.run(train_loader)
    metrics=train_evaluator.state.metrics
    accuracy=metrics['accuracy']
    loss=metrics['crossentropy']
    pbar.log_message(f'Training_loss: {loss}, Accuracy: {accuracy}')
    
@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_res():
    train_evaluator.run(validation_loader)
    metrics=train_evaluator.state.metrics
    accuracy=metrics['accuracy']
    loss=metrics['crossentropy']
    pbar.log_message(f'Validation_loss: {loss}, Accuracy: {accuracy}')
    
checkpoint=ModelCheckpoint('/tmp/model2/train_two','bert_model',n_saved=3,require_empty=False)
trainer.add_event_handler(Events.EPOCH_COMPLETED,checkpoint,{'bertmodel':model})

# Run Engine

In [14]:
trainer.run(train_loader,max_epochs=25)


Training_loss: 0.5648086929136936, Accuracy: 0.7573414446939761
Validation_loss: 0.6647356761871989, Accuracy: 0.7091534755677907



Training_loss: 0.48254929979105665, Accuracy: 0.7943826932370583
Validation_loss: 0.6244862461286993, Accuracy: 0.727598072952512



Training_loss: 0.4383963598373938, Accuracy: 0.8062938852090855
Validation_loss: 0.6273796401841019, Accuracy: 0.721403991741225



Training_loss: 0.40285256079967025, Accuracy: 0.8315660801091622
Validation_loss: 0.6334692097169649, Accuracy: 0.7362697866483139



Training_loss: 0.3819791862847173, Accuracy: 0.8485373965943657
Validation_loss: 0.6593020770603923, Accuracy: 0.7470061940812113


...


Training_loss: 0.08741543324651051, Accuracy: 0.9708332148847258
Validation_loss: 0.9260782513549553, Accuracy: 0.7478320715760496



Training_loss: 0.09396187494669812, Accuracy: 0.9629871791227222


2021-03-14 16:06:51,390 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


Validation_loss: 0.9661837846266346, Accuracy: 0.7343427391603579


State:
	iteration: 25300
	epoch: 23
	epoch_length: 1100
	max_epochs: 25
	output: 0.4693189859390259
	batch: <class 'tuple'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

# Predict Test set

In [15]:
predictions=[]
model.eval()
for batch in test_loader:
    with torch.no_grad():
        input_token,attention_mask=_batch(batch,train=False)
        out=model(input_token,attention_mask)
        predicted = torch.max(out,1)[1]
        y=predicted.cpu().detach().numpy()
        predictions.append(y)


In [16]:
predictions = np.hstack(predictions)
predictions=np.transpose(predictions)

In [17]:
ss=pd.read_csv(os.path.join(E.data_path,'SampleSubmission.csv'))
test=pd.read_csv(os.path.join(E.data_path,'Test.csv'))
ss.ID=test.ID
ss.label=predictions
mapping={0: 0, 1: 1, -1: 2}
inverse_mapping={x:y for y,x in mapping.items()}
ss['label']=ss.label.map(inverse_mapping)
ss.label.value_counts()
ss.to_csv('../Data/nb_3_ss_{}.csv',index=False)

In [18]:
ss.label.value_counts()

 1    16039
-1    13057
 0      904
Name: label, dtype: int64